In [1]:
from datasets.towncentre import load_towncentre

In [2]:
(xtr, ytr, img_names_tr), (xval, yval, img_names_val), (xte, yte, img_names_te) = load_towncentre('../data/TownCentre.pkl.gz',
                                                              canonical_split=True,
                                                              verbose=1)

Number of train samples: 6916
Number of validation samples: 874
Number of test samples: 904


In [3]:
import numpy as np
from utils import angles

def convert_and_pad_angle_data(y):
    """ In case there is only 1 component, we augment data with fake tilt and roll components
    """
    y_pan_bit = angles.deg2bit(y)
    y_tilt_bit = angles.deg2bit(np.zeros_like(y))
    y_roll_bit = angles.deg2bit(np.zeros_like(y))
    
    return np.hstack([y_pan_bit, y_tilt_bit, y_roll_bit])

In [4]:
ytr = convert_and_pad_angle_data(ytr)
yval = convert_and_pad_angle_data(yval)
yte = convert_and_pad_angle_data(yte)

In [5]:
from models.infinite_mixture import BiternionMixture

model = BiternionMixture(z_size=2, input_shape=xtr.shape[1:], 
                         backbone_cnn='mobilenet', backbone_weights=None, debug=False,
                         hlayer_size=512, n_samples=2)

/Users/sergeyprokudin/deep_direct_stat/py_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/sergeyprokudin/deep_direct_stat/py_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [ ]:
ckpt_path = '../logs/towncentre.h5'

model.fit(xtr, ytr, validation_data=[xval, yval], ckpt_path=ckpt_path, epochs=50, patience=5)